DBSCAN

In [ ]:
import pandas as pd
from utils.cluster import reassign_clusters, cluster, vectorize, tokenize,Cluster

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
dataset = ''
print('-' * 50)
print(f'Clustering {dataset} dataset...')

# load the dataset
df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
logs = df['Content'].tolist()
templates = df['EventTemplate'].tolist()

# tokenize -> vectorize -> cluster -> reassign_clusters
tokenized_logs = [tokenize(log) for log in logs]
labels, cluster_nums = cluster(vectorize(tokenized_logs))
labels, cluster_nums = reassign_clusters(labels, cluster_nums, tokenized_logs)

Show Clusters

In [ ]:
print('num of clusters:', cluster_nums)
print('len of templates:', len(set(templates)))

# store the logs in the cluster
inputs = []
for i in range(cluster_nums):
    inputs.append([-1, [], [], '']) # label, logs, indexs, ground_truth
for i, label in enumerate(labels):
    inputs[label][0] = label
    inputs[label][1].append(logs[i])
    inputs[label][2].append(i)
    if inputs[label][3] == '':
        inputs[label][3] = df['EventTemplate'][i]

clusters = []
for input in inputs:
    c = Cluster(*input, remove_duplicate= True)
    clusters.append(c)

num = 1
print('cluster:', num)
print('length:', len(clusters[num].indexs))
print('template:', clusters[num].oracle_template)
set_logs = set(clusters[num].static_logs)
print('len of set:', len(set_logs))
print('-'*20)
for log in set_logs:
    print(log)
print('='*40)

Evaluate

In [ ]:
from evaluate import evaluate_all_datasets
from IPython.display import HTML

theme = 'LogBatcher_0shot_32candidate_10batchsize'
table = evaluate_all_datasets(
    'LogBatcher_0shot_32candidate_10batchsize')
HTML(table)

Invocation Test

In [ ]:
from openai import OpenAI
from together import Together
import httpx
import openai
import backoff
from utils.postprocess import post_process

api_key = "sk-proj-5EkdZfTfjJ1GJim17pgQT3BlbkFJHCMqWAOX7dTSGOcFOjrn"
client = OpenAI(
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # proxies
    ),
)

# api_key = "3e0eca67e7a8720908ba848545b9175f556a6ab6dfc0b63ee3639f9d71e1bbeb"
# client = Together(
#     api_key=api_key,                      # api_key
# )


@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=20)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=0.0,
    )
    print(f"model: {response.model}")
    print(f"usage: {response.usage}")
    return response.choices[0].message.content.strip('\n')


output = get_responce([])

Check out some strings' freq in the whole logs or templates

In [ ]:
import pandas as pd
from utils.cluster import tokenize
from utils.sample_byword import extract_variables
from utils.postprocess import correct_single_template
from utils.postprocess import extract_variables

datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'HPC', 'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']

count = 0
count_templates = []
count_varaibles = {}
for dataset in datasets:
    # print(f"Processing {dataset} ----------------")
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log, template in zip(logs, templates):
        num = template.count('<*>')
        if count_varaibles.get(num):
            count_varaibles[num] += 1
        else:
            count_varaibles[num] = 1
print(count_varaibles)
for key in count_varaibles:
    count += count_varaibles[key]
for i in range(100):
    if count_varaibles.get(i):
        print(f"Number of templates with {i} variables: {count_varaibles[i]}")
print((4131+9825+7200+3700+2304+591) / 30000)
print((4131+9825+7200+3700+2304+591 + 563 + 1133 + 52 + 16 + 35) / 30000)

## sample based on entropy

In [ ]:
from utils.sample import sample_based_on_entropy

# datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
#         'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# for dataset in datasets:
#     pair = sample_based_on_entropy(dataset, 1)
#     print(pair[0][0])
dataset = 'HDFS'
pairs = sample_based_on_entropy(dataset, 1)
for pair in pairs:
    print(f"{pair[0][0]}\n{pair[0][1]}\n{'-'*20}")

## Caculate Cost

In [ ]:
import json
from utils.util import count_message_tokens, count_message_variables


table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
logs = {}
counts_token = {}
counts_message = {}
for dataset in datasets:
    df = pd.read_csv(f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv')
    logs[dataset] = df['Content'].tolist()
    counts_token[dataset] = 0
    counts_message[dataset] = 0
for log in logs['HealthApp']:
    log = log.strip()

# 存储解析后的日志列表
message_list = []
# load every message
file = 'cost_lilac_32_3.json'
with open('outputs/cost/LogBatcher_3shot_32candidate_10batchsize.json', 'r') as file:
    for line in file:
        if line.strip() == '[':
            list_str = ''
            start_load = True
        if line.strip() == ']':
            list_str += line
            message = json.loads(list_str)
            message_list.append(message)
            start_load = False
        if start_load:
            list_str += line
# print(len(message_list))
for message in message_list:
    # for LILAC
    log = message[-1]['content'].split('\n')[0].replace('Log message: `', '').replace('`', '')
    # for LogBatcher
    # log = message[-1]['content'].split('\n')[0] 
    # print(log)
    for dataset in datasets:
        if log in logs[dataset]:
            counts_token[dataset] += count_message_tokens(message, 'gpt-3.5-turbo')
            counts_message[dataset] += 1
            break
        if dataset == 'Mac':
            print(log)
for dataset in datasets:
    print(counts_token[dataset], counts_token[dataset] / counts_message[dataset]  )

print(sum(counts_token.values()), sum(counts_token.values()) / sum(counts_message.values()))

# remove the same log messages

# def make_hashable(log_list):

#     return tuple(tuple(sorted(d.items())) for d in log_list)
# unique_lists = list(set(make_hashable(log_list) for log_list in message_list))

# unique_big_list = [list(map(dict, log_list)) for log_list in unique_lists]
# print(len(unique_big_list))

Unseen logs

In [ ]:
from collections import Counter
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cosine
import numpy as np

table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
a = 0
b = 0
for dataset in datasets:

    file = 'Test_10shot'
    groundtruth_file = f'dataset/{dataset}/{dataset}_2k.log_structured_corrected.csv'
    output_file = f'outputs/parser/{file}/{dataset}_2k.log_structured.csv'

    templates_groundtruth = pd.read_csv(groundtruth_file)[
        'EventTemplate'].tolist()
    templates_output = pd.read_csv(output_file)['EventTemplate'].tolist()

    freq = Counter(templates_groundtruth)
    unseen_templates = [item for item, count in freq.items() if count == 1]

    indexes = []
    for index, template in enumerate(templates_groundtruth):
        if template in unseen_templates:
            indexes.append(index)

    accuracy_exact_string_matching = accuracy_score([templates_groundtruth[index] for index in indexes], [
                                                    templates_output[index] for index in indexes], normalize=False)
    length = len(indexes)
    a += length
    b += accuracy_exact_string_matching
    dataset = ' ' * (12 - len(dataset)) + dataset
    print('%s: len of unseen log: %.4d, Message-Level Accuracy: %.4f' %
          (dataset, length, accuracy_exact_string_matching/length))


print(b/a)
print(a/len(datasets))
# 81.0 71.2

Data load
main result

In [ ]:
c = '''0.998 	0.999 	0.999 	0.998 	0.999 	0.999 	0.998 	0.959 	0.997 	0.930 	0.961 	0.993 	0.930 	0.996 	0.999 	1.000 	1.000 	1.000 	1.000 	1.000 	1.000 
0.948 	0.613 	0.882 	0.869 	0.606 	0.901 	0.950 	0.158 	0.751 	0.694 	0.195 	0.708 	0.683 	0.744 	0.915 	0.991 	0.958 	0.986 	0.990 	0.886 	0.952 
0.920 	0.398 	0.963 	0.905 	0.398 	0.952 	0.998 	0.376 	0.950 	0.470 	0.296 	0.915 	0.738 	0.960 	0.983 	0.999 	0.983 	0.998 	0.998 	0.972 	0.989 
0.967 	0.799 	0.981 	0.965 	0.800 	0.981 	0.989 	0.779 	0.987 	0.956 	0.805 	0.970 	0.955 	0.979 	0.998 	1.000 	0.987 	0.999 	0.995 	0.988 	0.995 
0.963 	0.479 	0.885 	0.957 	0.474 	0.883 	0.996 	0.426 	0.891 	0.702 	0.282 	0.785 	0.736 	0.950 	0.990 	0.983 	0.972 	0.989 	0.987 	0.941 	0.989 
0.887 	0.662 	0.872 	0.904 	0.680 	0.880 	0.945 	0.660 	0.973 	0.978 	0.751 	0.870 	0.935 	0.980 	0.997 	0.970 	0.994 	0.999 	0.953 	0.943 	0.995 
0.957 	0.180 	0.941 	0.945 	0.180 	0.943 	0.971 	0.060 	0.932 	0.554 	0.097 	0.826 	0.234 	0.879 	0.978 	0.984 	0.913 	0.983 	0.914 	0.854 	0.953 
0.997 	0.466 	0.948 	0.691 	0.158 	0.840 	0.997 	0.463 	0.976 	0.694 	0.141 	0.915 	0.710 	0.715 	0.903 	0.696 	0.685 	0.897 	1.000 	0.609 	0.862 
0.422 	0.217 	0.750 	0.405 	0.205 	0.745 	0.358 	0.176 	0.770 	0.186 	0.125 	0.684 	0.484 	0.620 	0.935 	0.298 	0.422 	0.926 	0.998 	0.731 	0.969 
0.885 	0.750 	0.972 	0.773 	0.540 	0.876 	0.960 	0.253 	0.924 	0.795 	0.436 	0.822 	0.737 	0.677 	0.952 	0.953 	0.627 	0.923 	0.971 	0.787 	0.953 
0.901 	0.375 	0.749 	0.893 	0.368 	0.744 	1.000 	0.261 	0.871 	0.833 	0.677 	0.850 	0.876 	0.984 	0.997 	0.998 	0.988 	0.998 	0.920 	0.914 	0.961 
1.000 	0.978 	0.996 	1.000 	0.978 	0.996 	1.000 	0.984 	0.996 	1.000 	0.972 	0.995 	0.984 	0.985 	0.997 	1.000 	1.000 	1.000 	1.000 	0.978 	0.996 
0.789 	0.594 	0.919 	0.547 	0.729 	0.965 	1.000 	0.287 	0.948 	0.802 	0.928 	0.960 	0.749 	0.987 	0.999 	0.753 	0.805 	0.983 	1.000 	0.976 	0.989 
0.224 	0.105 	0.693 	0.249 	0.034 	0.718 	0.492 	0.112 	0.937 	0.315 	0.071 	0.724 	0.220 	0.437 	0.873 	1.000 	0.977 	0.991 	1.000 	0.982 	0.995 
0.814 	0.392 	0.896 	0.765 	0.284 	0.835 	0.949 	0.383 	0.902 	0.759 	0.359 	0.843 	0.712 	0.549 	0.898 	0.805 	0.562 	0.892 	0.831 	0.528 	0.879 
0.845 	0.534 	0.896 	0.791 	0.495 	0.884 	0.907 	0.422 	0.920 	0.711 	0.473 	0.857 	0.712 	0.829 	0.961 	0.895 	0.858 	0.971 	0.970 	0.873 	0.965'''

# count baseline num
num = int(len(c.split('\n')[0].split('\t')) / 3)
GA, MLA, ED = [], [], []
for i in range(num):
    GA.append([])
    MLA.append([])
    ED.append([])

lines = c.split('\n')
for line in lines:
    num_list = line.split('\t')
    for i in range(num):
        GA[i].append(float(num_list[i*3]))
        MLA[i].append(float(num_list[i*3 + 1]))
        ED[i].append(float(num_list[i*3 + 2]))

Robust

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

data = {
    'Log Parser': ['Drain', 'AEL', 'Brain', 'Logram', 'DivLog', 'LILAC', 'LogBatcher'],
    'Group Accuracy': GA,
    'Parsing Accuracy': MLA,
    'Normalized Edit Distance': ED
}

# 将数据转换为DataFrame
df = pd.DataFrame(data)
flierprops = dict(marker='o', color='black', markersize=3,
                  markerfacecolor='black', markeredgecolor='black')
# 绘制箱线图
fig, axs = plt.subplots(1, 3, figsize=(16, 5))

label_fontsize = 18
x_fontsize = 18
y_fontsize = 18
special_tick_fontsize = 16  # 特定指标的字体大小
space_size = 0.05
line_size = dict(linewidth=1.5)


for ax in axs[:2]:
    for i in [0, 0.2, 0.4, 0.6, 0.8, 1.0]:
        ax.axhline(y=i, color='gray', linestyle='--',
                   linewidth=1, dashes=(5, 5))
    ax.tick_params(axis='y', labelsize=y_fontsize)

for i in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    axs[2].axhline(y=i, color='gray', linestyle='--',
                   linewidth=1, dashes=(5, 5))
# Adjust y-axis label font siz
axs[2].tick_params(axis='y', labelsize=y_fontsize)
# Group Accuracy
axs[0].boxplot(df['Group Accuracy'], labels=df['Log Parser'], boxprops=line_size, capprops=line_size,
               whiskerprops=line_size, medianprops=line_size,
               flierprops=flierprops)
axs[0].set_ylabel('Group Accuracy', rotation=90, fontsize=label_fontsize)
axs[0].set_ylim(0-space_size, 1+space_size)
axs[0].set_xticklabels(df['Log Parser'], rotation=45,
                       ha='right', fontsize=x_fontsize)


# Parsing Accuracy
axs[1].boxplot(df['Parsing Accuracy'], boxprops=line_size, capprops=line_size,
               whiskerprops=line_size, medianprops=line_size,
               labels=df['Log Parser'], flierprops=flierprops)
axs[1].set_ylabel('Parsing Accuracy', rotation=90, fontsize=label_fontsize)
axs[1].set_ylim(0-space_size, 1+space_size)
axs[1].set_xticklabels(df['Log Parser'], rotation=45,
                       ha='right', fontsize=x_fontsize)

# Edit Distance
axs[2].boxplot(df['Normalized Edit Distance'], boxprops=line_size, capprops=line_size,
               whiskerprops=line_size, medianprops=line_size,
               labels=df['Log Parser'], flierprops=flierprops)
axs[2].set_ylabel('Edit Distance', rotation=90, fontsize=label_fontsize)
axs[2].set_ylim(0.5 - space_size/2, 1 + space_size/2)
axs[2].set_xticklabels(df['Log Parser'], rotation=45,
                       ha='right', fontsize=x_fontsize)

plt.tight_layout()

# 保存为 PDF 文件
plt.savefig('outputs/figures/RQ1_Robust.pdf', format='pdf')

plt.show()

Supervised Figures

In [ ]:
import matplotlib.pyplot as plt

# Load data

# LILAC
data1 = '''0.8298 0.6569 0.91518
0.8852 0.8023 0.95239
0.8894 0.8418 0.9652
0.8952 0.858 0.97086
0.8947 0.8313 0.96806
0.8953 0.8624 0.97133
0.8965 0.8637 0.96976'''

# LogBatcher
data2 = '''0.970 	0.873 	0.965 
0.973 	0.942 	0.985 
0.979 	0.967 	0.992 
0.975 	0.956 	0.989 
0.979 	0.939 	0.980 
0.979 	0.954 	0.986 
0.980 	0.928 	0.976'''

# Function to process data


def process_data(data):
    accuracy_metric1 = []
    accuracy_metric2 = []
    accuracy_metric3 = []

    data_list = data.replace('\t', ' ').replace(
        '\n', ' ').replace('  ', ' ').split()
    for index, data_item in enumerate(data_list):
        if index % 3 == 0:
            accuracy_metric1.append(float(data_item))
        elif index % 3 == 1:
            accuracy_metric2.append(float(data_item))
        else:
            accuracy_metric3.append(float(data_item))

    return accuracy_metric1, accuracy_metric2, accuracy_metric3


# Process the data
accuracy_metric1_data1, accuracy_metric2_data1, accuracy_metric3_data1 = process_data(
    data1)
accuracy_metric1_data2, accuracy_metric2_data2, accuracy_metric3_data2 = process_data(
    data2)

# Data for the plot
shots = [0, 1, 2, 3, 4, 5, 6]

# Create the subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

font_size = 36
label_size = 30
title_size = 32
legend_size = 20

# Plot data1
ax1.plot(shots, accuracy_metric1_data1, marker='o', linestyle='--', label='GA')
ax1.plot(shots, accuracy_metric2_data1,
         marker='s', linestyle='--', label='MLA')
ax1.plot(shots, accuracy_metric3_data1,
         marker='^', linestyle='--', label='ED')
ax1.set_xlabel('demonstration number', fontsize=font_size)
ax1.set_xticks(shots)
ax1.set_yticks([i/10 for i in range(5, 11)])
ax1.set_ylim(0.6, 1)
ax1.legend(loc='lower right', prop={'size': 20})
ax1.grid(True)
ax1.set_title('LILAC', fontsize=title_size)

# Set bold font for tick labels on ax1
plt.setp(ax1.get_xticklabels(), fontsize=label_size)
plt.setp(ax1.get_yticklabels(), fontsize=label_size)

# Plot data2
ax2.plot(shots, accuracy_metric1_data2, marker='o', linestyle='--', label='GA')
ax2.plot(shots, accuracy_metric2_data2,
         marker='s', linestyle='--', label='MLA')
ax2.plot(shots, accuracy_metric3_data2,
         marker='^', linestyle='--', label='ED')
ax2.set_xlabel('demonstration number', fontsize=font_size)
ax2.set_xticks(shots)
ax2.set_yticks([i/10 for i in range(5, 11)])
ax2.set_ylim(0.6, 1)
ax2.legend(loc='lower right', prop={'size': 20})
ax2.grid(True)
ax2.set_title('LogBatcher', fontsize=title_size)

# Set bold font for tick labels on ax2
plt.setp(ax2.get_xticklabels(), fontsize=label_size)
plt.setp(ax2.get_yticklabels(), fontsize=label_size)

# Create a common legend for both plots
# handles, labels = ax1.get_legend_handles_labels()
# fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(
#     0.5, 1.15), ncol=3, prop={'size': legend_size})

plt.subplots_adjust(wspace=30)

# Show the plot
plt.tight_layout()
plt.savefig('outputs/figures/RQ3_supervised.pdf')
plt.show()

Ablation: Bacth Size

In [ ]:
import matplotlib.pyplot as plt

# Define the data
data1 = '''0.923 0.708 0.905
0.971 0.848 0.953 
0.970 0.873 0.965 
0.940 0.816 0.948 
0.928 0.804 0.942'''

# Function to process data


def process_data(data):
    accuracy_metric1 = []
    accuracy_metric2 = []
    accuracy_metric3 = []

    data_list = data.replace('\n', ' ').strip().split()
    for index, data_item in enumerate(data_list):
        if index % 3 == 0:
            accuracy_metric1.append(float(data_item))
        elif index % 3 == 1:
            accuracy_metric2.append(float(data_item))
        else:
            accuracy_metric3.append(float(data_item))

    return accuracy_metric1, accuracy_metric2, accuracy_metric3


# Process the data
accuracy_metric1_data1, accuracy_metric2_data1, accuracy_metric3_data1 = process_data(
    data1)

# Data for the plot
shots = [1, 5, 10, 15, 20]

# Create the plot
fig, ax1 = plt.subplots(figsize=(8, 6))

font_size = 36
label_size = 24
title_size = 32

# Plot data1
ax1.plot(shots, accuracy_metric1_data1, marker='o',
         linestyle='--', label='GA', linewidth=2, markersize=10)
ax1.plot(shots, accuracy_metric2_data1, marker='s',
         linestyle='--', label='MLA', linewidth=2, markersize=10)
ax1.plot(shots, accuracy_metric3_data1, marker='^',
         linestyle='--', label='ED', linewidth=2, markersize=10)
ax1.set_xlabel('batch size', fontsize=font_size)
ax1.set_xticks(shots)
ax1.set_yticks([i/100 for i in range(30, 101, 5)])
ax1.set_ylim(0.65, 1)
ax1.legend(loc='lower right', prop={'size': 20})
ax1.grid(True)
# ax1.set_title('LILAC', fontsize=title_size)

# Set bold font for tick labels on ax1
plt.setp(ax1.get_xticklabels(), fontsize=label_size)
plt.setp(ax1.get_yticklabels(), fontsize=label_size)

# Show the plot
plt.tight_layout()
plt.savefig('outputs/figures/RQ2_batchsize.pdf')
plt.show()